# Create and Populate PostgreSQL Output Database for FVS Simulations
We're using a PostgreSQL database to collect outputs from the Forest Vegetation Simulator (FVS) growth-and-yield model to allow for running multiple FVS simulations in parallel. PostgreSQL allows multiple simultaneous connections to the database, so FVS won't have to wait for other running instances of FVS to finish reading/writing. 

In [ ]:
import psycopg2 # for interacting with PostgreSQL database
from FVSoutput_SQL_createDBtables import * # SQL query strings for creating FVS output tables

## Getting Started

First, you'll need to create a blank output database you want FVS to direct outputs to. Don't forget to add the output database information to your list of ODBC named data sources so that FVS knows how to find and access it. The rest of this notebook will still work and your output tables will get created, but FVS won't write to your output database if it doesn't find it as a named data source (with 32-bit ODBC driver, suggest using PostgreSQL UNICODE).

## Tables you can create in the output database

The following tables are available to use. All should be pre-prended with "fvs_": 

**Base FVS (incl. DB Extension):**  
atrtrlist, cases, compute, cutlist, strclass, summary, treelist

**Fire & Fuels Extension (FFE):**  
*General:* canprofile, snagdet, down_wood_cov, down_wood_vol, mortality, snagsum  
*Fire/Fuels:* burnreport, consumption, potfire, fuels  
*Carbon:* carbon, hrv_carbon, 



**ECON Extension:**  
econharvestvalue, econsummary

**Mountain Pine Beetle (MPB) Impact Model:**  
bm_bkp, bm_main, bm_tree, bm_vol

**Climate-FVS:**  
climate  

**Dwarf Mistletoe:**  
dm_spp_sum, dm_stnd_sum  

**Western Root Disease:**  
rd_sum, rd_det, rd_beetle

In [ ]:
# We'll use the following helper function to write tables into the FVS Output database.
def create_tables(conn_str, table_SQLs, verbose=False):
    '''
    Creates tables in a PostgreSQL database to hold FVS Outputs.
    ===========
    Arguments:
    conn_str = A string for the database connection. 
    table_SQLs = A list of valid FVS table names (imported from FVSoutput_SQL_createDBtables.py).
    verbose = Will print out the SQL query strings used if set to True (defaults to False).
    '''
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            for SQL in table_SQLs:
                if verbose:
                    print(SQL)
                cur.execute(SQL)
                print('Created', SQL.split(' ')[2], end='... ')
    conn.close()
    print('Done.')

### Specify the tables you want to be created in the output database

In [ ]:
my_tables = [fvs_cases, fvs_summary, fvs_carbon, fvs_hrv_carbon, fvs_econharvestvalue, fvs_econsummary]

### Specify how to connect to your output database
Specify the dbname, username/owner, and host for your database. If you don't have password authentication handled via other means, you'd also need to provide it here.

In [ ]:
mydb = "FVSOut"
myusername = 'postgres'
myhost = 'localhost'
conn_string = "dbname={dbname} user={user} host={host}".format(dbname=mydb, user=myusername, host=myhost)

### Execute the `create_tables` function, then go check out your database!

In [ ]:
create_tables(conn_string, my_tables)